In [1]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import time
from jupyter_util import *
from importlib import reload

In [2]:
# Connect agents
agents = [8150,8151]
connection = [1,2]
inv = [0,1]
get_index = lambda x: 1 if x == 0 else 0

for i in range(len(connection)):
    inv[i] = post(agents[i], "/connections/create-invitation?alias=agent_connection&auto_accept=true&multi_use=true")
    # Create then register DIDS 
    resp = post(agents[i],  "/wallet/did/create")
    dids = requests.post("http://localhost:9000/register", json={
        "did": resp['result']['did'], 
        "verkey":resp['result']['verkey'], 
        "alias": "BOB"+str(agents[i]), 
        "role": "ENDORSER"
    })
    resp = post(agents[i], "/wallet/did/public?did=" + resp['result']['did'])

for i in range(len(connection)):
    connection[i] = post(agents[i], "/connections/receive-invitation?auto_accept=true", {
        "label": "Bob",
        "recipientKeys": inv[get_index(i)]['invitation']['recipientKeys'],
        "serviceEndpoint": inv[get_index(i)]['invitation']['serviceEndpoint'],
    })['connection_id']

# Services

In [3]:
set_pds_config_agent1(
    "own_your_data",
    {
        "client_id": "gVQJTSabNK8DxNzu3PeGWQwBtXSb2Iv7FgYlbzBEbfg",
        "client_secret": "9yBmtiyO2YwhtHA9btSrYensVZZ9DLNY1Vq5D-EjxwQ",
        "api_url": "https://data-vault.eu",
    },
)

set_pds_config_agent2(
    "own_your_data",
    {
        "client_id": "nM8p2yYuxqGWvbJ50t8ODifjyFZfi-yOm2HoE6AZaW0",
        "client_secret": "X6WBjmMylTQ3BdbBRKkBir3uVz79RgADetw5tl7Tgyo",
        "api_url": "https://data-vault.eu",
    },
)

# set_pds_config_agent2(
#     "own_your_data",
#     { "scope": "admin", "client_id": "50c67447e28eeb5ac56bd8bc3fbfbc5916f817603eb3f117f01e84d52e36afb7", "client_secret": "6f20318f0a451b24a9c4aaa00214626d6e0f6b15d18957acd11a085c4d7b2dbd", "api_url": "https://dip-officer.data-container.net" }
# )


{"success": "True", "status": {"own_your_data": {"connected": true}}}
{"success": "True", "status": {"own_your_data": {"connected": true}}}


In [4]:
import random
add_consent = post(8151, "/verifiable-services/consents", {
    "label": "test_" + str(random.randint(1, 100000000)),
    "oca_schema_namespace": "consent",
    "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s",
    "oca_data": {
        "expiration": "7200",
        "limitation": "7200",
        "dictatedBy": "test",
        "validityTTL": "7200",
    }
})
get_consents_provider = get(8151, "/verifiable-services/consents")
get_consents_user = get(8150, "/verifiable-services/consents")

save_data_to_pds = post(8151, "/pds/save", {"payload": json.dumps({
        "associatedReportID": "Text12",
        "uniqueNumericIDHash": "Text",
        "disease": "Text",
        "vaccineDescription": "Text",
        "atcCode": "Text",
        "certificateIssuer": "Text",
        "certificateNumber": "Text",
        "certificateValidFrom": "Date",
        "certificateValidTo": "Date",
        "formVersion": "Text",
    }), "oca_schema_dri": "dip.data.tda.oca_chunks.predefined.9GdWoQwth9299oYj8HfdgRZjtSxW9sTVyy1RnfhQ35yA"
})

In [5]:
add = post(8151, '/verifiable-services/add', {
  "label": "consent_test",
  "service_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "12345"
  },
  "certificate_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "9GdWoQwth9299oYj8HfdgRZjtSxW9sTVyy1RnfhQ35yA"
  },
  "consent_id": add_consent['consent_id']
})

In [6]:
service_list = get(8150, '/verifiable-services/DEBUGrequest/' + connection[0])
webhook_service_list = get(8150, '/verifiable-services/request-service-list/' + connection[0])

In [7]:
service = service_list['result']['services'][-1]

In [8]:
# service = None
# services = service_list['result']['services']
# for i in services:
#     if i['service_id'] == add['service_id']:
#         service = i
# assert service == add['service_id']

In [9]:
service = service_list['result']['services'][0]
service.pop("policy_validation", None)
apply = post(8150, '/verifiable-services/apply', {
  "connection_id": connection[0],
  "user_data": '{"memes": "films"}',
  "service": service,
})
print(apply)

{'success': True, 'exchange_id': 'f1f93903-4c2e-46e1-b243-4a69bc0eafa0'}


In [10]:
time.sleep(1)
issue_self = requests.post('http://localhost:8151/verifiable-services/get-issue', json={
    "exchange_id": apply['exchange_id'],
})
issue_self = json.loads(issue_self.text)
issue_self = issue_self['result']

In [11]:
process_application = post(8151, '/verifiable-services/process-application', {
    "issue_id": issue_self[0]['issue_id'],
    "decision": "accept",
    "data": {
        "test": "certificate"
    }
})


In [12]:
issue_self = post(8151, '/verifiable-services/get-issue', {
    "exchange_id": apply['exchange_id'],
})['result'][0]
issue_self['consent_schema'].pop("usage_policy")
print(issue_self['state'])

accepted


In [15]:
issue_self = post(8150, '/verifiable-services/get-issue', {
    "exchange_id": apply['exchange_id'],
})['result']
print(issue_self[0]["state"])

credential_received


In [16]:
cred = get(8151, "/pds/" + issue_self[0]["credential_id"])
user_data = get(8151, "/pds/" + issue_self[0]["service_user_data_dri"])
consent = get(8151, "/pds/" + issue_self[0]["user_consent_credential_dri"])
report = get(8151, "/pds/" + issue_self[0]["report_data_dri"])
cred_content = get(8151, "/pds/" + cred['payload']['credentialSubject']['oca_data_dri'])
report = get(8151, "/verifiable-services/report/" + cred_content['payload']['associatedReportID'])

In [17]:
cred = get(8150, "/pds/" + issue_self[0]["credential_id"])
user_data = get(8150, "/pds/" + issue_self[0]["service_user_data_dri"])
consent = get(8150, "/pds/" + issue_self[0]["user_consent_credential_dri"])
report = get(8150, "/pds/" + issue_self[0]["report_data_dri"])
cred_content = get(8150, "/pds/" + cred['payload']['credentialSubject']['oca_data_dri'])
report = get(8150, "/verifiable-services/report/" + cred_content['payload']['associatedReportID'])

In [18]:
credentials = requests.get('http://localhost:8150/credentials')
credentials = json.loads(credentials.text) 
pp.pprint(credentials)

{'result': [{'content': {'context': ['https://www.w3.org/2018/credentials/v1',
                                     'https://www.schema.org'],
                         'credentialSubject': {'oca_data_dri': 'zQmeJXcPVVh9A3uw5Hm3QN2TwV5tnzJgFDSErDSttrXtzci',
                                               'oca_schema_dri': '9GdWoQwth9299oYj8HfdgRZjtSxW9sTVyy1RnfhQ35yA',
                                               'oca_schema_namespace': 'string',
                                               'service_consent_match_id': '75b35a9a-6e40-47d0-bd30-dd2fbb6ac5ef',
                                               'subject_id': 'F5v83yGrTLb5i3WsifSpDF'},
                         'issuanceDate': '2021-05-09 09:26:21.999383Z',
                         'issuer': 'DrXfjiPYrqLdwwY95XSXhr',
                         'proof': {'created': '2021-05-09 09:26:22.005656Z',
                                   'jws': 'dCWhEto4k2QrLcnY193SzxJ3Nloxm-9mbo8ke0vpfZ5HTAT-kQjsVf5FBvcs1FA_lEq1gWBdIBYc4x8syPvABw',
    

In [19]:
credential = credentials['result'][-2]['content']
print(credential['credentialSubject']['oca_schema_dri'])

9GdWoQwth9299oYj8HfdgRZjtSxW9sTVyy1RnfhQ35yA


In [21]:
request_proof = post(
    8151,
    "/present-proof/request",
    {
        "connection_id": connection[1],
        "schema_base_dri": credential['credentialSubject']['oca_schema_dri'],
        "requested_attributes": ["address"],
        "issuer_did": "123145",
    },
)
exchange_id_agent2 = request_proof["exchange_id"]
exchange_record = {"result": []}
while exchange_record.get("result") == []:
    exchange_record = get(
        8150,
        f"/present-proof/exchange/record?state=request_received",
    )

# # FETCH CREDENTIALS (1)
creds = get(8150, "/credentials")

# # PRESENT PROOF (1)
present = post(
    8150,
    "/present-proof/present",
    {
        "credential_id": creds["result"][0]["dri"],
        "exchange_record_id": exchange_record["result"][0][
            "presentation_exchange_id"
        ],
    },
)

exchange_record = get(
    8151,
    f"/present-proof/exchange/record?connection_id={connection[1]}&thread_id={request_proof['thread_id']}",
)

time.sleep(1)
exchange_record = get(
    8151,
    f"/present-proof/exchange/record?state=presentation_received",
)

post(
    8151,
    f"/present-proof/acknowledge?status=true&exchange_record_id={exchange_record['result'][0]['presentation_exchange_id']}&person_id=123fqef9yu",
)


{'success': True,
 'message': 'ack sent and exchange record updated',
 'exchange_record_id': '15aeb748-8f94-48ee-9e66-799688f1ee85',
 'ack_credential_dri': 'zQmWmrd596ea5woqUob9BGLnsgSZAUWxxZM2CkKyPMQrWqJ'}

In [23]:
request_proof = post(
    8151,
    "/present-proof/request",
    {
        "connection_id": connection[1],
        "schema_base_dri": credential['credentialSubject']['oca_schema_dri'],
        "requested_attributes": ["address"],
        "issuer_did": "123145",
    },
)
exchange_id_agent2 = request_proof["exchange_id"]
exchange_record = {"result": []}
while exchange_record.get("result") == []:
    exchange_record = get(
        8150,
        f"/present-proof/exchange/record?state=request_received")
post(
    8150,
    "/present-proof/reject",
    {
        "exchange_record_id": exchange_record["result"][0][
            "presentation_exchange_id"
        ],
    },
)

{'exchange_id': 'ba598f43-3cf1-4e57-a837-a121995e2211'}

In [ ]:
connection[0]